In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from importlib import reload
import json
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import MultiLearn_GLRM

In [64]:
df_train = pd.read_csv('./microarray/xtrain.data', sep='\s+', header=None)
df_label_train = pd.read_csv('./microarray/ytrain.data', sep='\s+', header=None)
df_test = pd.read_csv('./microarray/xtest.data', sep='\s+', header=None)
df_label_test = pd.read_csv('./microarray/ytest.data', sep='\s+', header=None)

df_train.loc[-1] = df_label_train.loc[0].to_list()
df_train.index = df_train.index + 1  
df_train = df_train.sort_index()
df_train = df_train.T

df_test.loc[-1] = df_label_test.loc[0].to_list()
df_test.index = df_test.index + 1  
df_test = df_test.sort_index()
df_test = df_test.T
df_test = df_test.dropna()

X_train = df_train.drop(columns=[0])
Y_train = df_train[0]
Y_train = np.array(Y_train, dtype=np.int64).reshape(-1,1)

X_test = df_test.drop(columns=[0])
Y_test = df_test[0]
Y_test = np.array(Y_test, dtype=np.int64).reshape(-1,1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


params = {
    "train_hyper_params": None,
    "test_hyper_params": None,
    "beta": None,
    "embedding_matrix": None,
    "train_error": None,
    "test_error": None,
    "embedding_dim": None,
    "predictor_scaling_parmas": None,
    "target_scaling_parmas": None
}

In [69]:
df_train.drop(columns=[0]).to_csv()

,1,2,3,4,5,6,7,8,9,10,...,2299,2300,2301,2302,2303,2304,2305,2306,2307,2308
0,0.773344,-2.438405,-0.482562,-2.721135,-1.217058,0.827809,1.342604,0.057042,0.133569,0.565427,...,-0.238511,-0.027474,-1.660205,0.588231,-0.463624,-3.952845,-5.496768,-1.414282,-0.647600,-1.763172
1,-0.078178,-2.415754,0.412772,-2.825146,-0.626236,0.054488,1.429498,-0.120249,0.456792,0.159053,...,-0.657394,-0.246284,-0.836325,-0.571284,0.034788,-2.478130,-3.661264,-1.093923,-1.209320,-0.824395
2,-0.084469,-1.649739,-0.241308,-2.875286,-0.889405,-0.027474,1.159300,0.015676,0.191942,0.496585,...,-0.696352,0.024985,-1.059872,-0.403767,-0.678653,-2.939352,-2.736450,-1.965399,-0.805868,-1.139434
3,0.965614,-2.380547,0.625297,-1.741256,-0.845366,0.949687,1.093801,0.819736,-0.284620,0.994732,...,0.259746,0.357115,-1.893128,0.255107,0.163309,-1.021929,-2.077843,-1.127629,0.331531,-2.179483
4,0.075664,-1.728785,0.852626,0.272695,-1.841370,0.327936,1.251219,0.771450,0.030917,0.278313,...,-0.200404,0.061753,-2.273998,-0.039365,0.368801,-2.566551,-1.675044,-1.082050,-0.965218,-1.836966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,-2.683846,-2.117767,-0.523236,-2.174192,-0.448947,0.543022,1.338731,0.194250,0.599276,0.637370,...,-1.681471,-1.036765,-0.422204,-1.400799,-2.831914,-4.667046,-2.354932,-1.220102,-0.435100,-1.444771
59,-1.207646,-2.478130,0.094310,0.273456,-1.113218,-0.283690,1.105985,0.801284,0.381582,0.714811,...,-0.929629,-0.258512,-1.482365,-0.827136,-1.543182,-1.549755,-0.532730,-1.759099,0.570471,-2.232127
60,-1.684161,-2.053384,0.563835,0.314446,-2.052605,-0.630172,1.570967,0.137324,1.036985,0.973654,...,-0.537683,-0.497909,-0.477552,-1.427950,-1.928955,-2.544657,-1.080576,-1.382302,-0.804973,-1.318262
61,-2.258568,-2.308603,-1.443076,0.233094,-1.779633,-0.030665,1.532665,-0.408419,0.795162,0.376242,...,-2.276917,-0.245517,-0.815315,-1.288079,-2.835316,-2.169804,-0.630172,-0.870839,-0.315533,-1.326140


In [46]:
params["predictor_scaling_parmas"] =  {
    "mean" : scaler.mean_.tolist(),
    "var" : scaler.mean_.tolist()
}

In [65]:
n_class = max(max(Y_train), max(Y_test))[0]

one_hot_targets = np.eye(n_class)[Y_train.reshape(-1,)-1]
Train = np.hstack((X_train, one_hot_targets))

In [66]:
Train.shape, X_train.shape

((63, 2312), (63, 2308))

In [67]:
reload(MultiLearn_GLRM)
from MultiLearn_GLRM import Multi_Learn, generate_AXY
from MultiLearn_GLRM.Params.semisupervised_params import Multilearn_GLRM_Semisupervised_Train_Params, Multilearn_GLRM_Semisupervised_Test_Params

embedding_dim = 30
params["embedding_dim"] = embedding_dim

n_class = n_class

A_prime, X_prime, Y_prime = generate_AXY.get_semisupervised_glrm_train_form(Train[:,:-n_class], Train[:,-n_class:], n_class, embedding_dim)
GLRM_loss_list, X_regulariation_list, Y_regulariation_list, X_grad_restrictions, Y_grad_restrictions = Multilearn_GLRM_Semisupervised_Train_Params(A_prime, X_prime, Y_prime, n_class)

params["train_hyper_params"] = [GLRM_loss_list, X_regulariation_list, Y_regulariation_list]

num_iterations=1000
learning_rate=0.5
result = Multi_Learn.alternating_minimization(A_prime, X_prime, Y_prime, GLRM_loss_list, X_regulariation_list, Y_regulariation_list, X_grad_restrictions, Y_grad_restrictions, num_iterations, learning_rate, n_class)


epoch: 0
iter: 1 Total loss: tf.Tensor(463.02745665009087, shape=(), dtype=float64) [9.33952975168517, 31.856640809686052]
iter: 8 Total loss: tf.Tensor(442.40353629796334, shape=(), dtype=float64)
iter: 1 Total loss: tf.Tensor(442.3910704155646, shape=(), dtype=float64)
iter: 100 Total loss: tf.Tensor(66.73679990263159, shape=(), dtype=float64)
iter: 133 Total loss: tf.Tensor(61.95130711766721, shape=(), dtype=float64)
epoch: 1
iter: 1 Total loss: tf.Tensor(63.36377078954476, shape=(), dtype=float64) [6.509962989487958, 1.0622421615025643]
iter: 6 Total loss: tf.Tensor(63.2918471250554, shape=(), dtype=float64)
iter: 1 Total loss: tf.Tensor(63.29745567786591, shape=(), dtype=float64)
iter: 100 Total loss: tf.Tensor(83.48354257021833, shape=(), dtype=float64)
iter: 199 Total loss: tf.Tensor(38.30020703171905, shape=(), dtype=float64)
iter: 222 Total loss: tf.Tensor(59.503649913865026, shape=(), dtype=float64)
epoch: 2
iter: 1 Total loss: tf.Tensor(48.710380763184915, shape=(), dtype=fl

KeyboardInterrupt: 

In [ ]:
beta = result[1][1:,0:n_class]
Y_final = result[1][1:embedding_dim+1,n_class:]
params["beta"] = beta.numpy().tolist()
params["embedding_matrix"] = Y_final.numpy().tolist()

train_prob = tf.nn.softmax(np.matmul(result[0], result[1][:,0:n_class])).numpy()
label_train_shuffled =  np.argmax(Train[:,-n_class:], axis=1)
params["train_accuracy"] = accuracy_score(label_train_shuffled, np.argmax(train_prob, axis=1))
params["train_accuracy"]

In [58]:
label_train_shuffled

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0])

In [59]:
np.argmax(train_prob, axis=1)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0])

In [60]:
one_hot_targets_val = np.eye(n_class)[Y_test.reshape(-1,)-1]
one_hot_targets_val

array([[0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [ ]:
# res = np.matmul(result[0], result[1][:,0:n_class])
# np.mean((Y_train - res)**2)
# 0.4495674496068231

# res = np.matmul(result[0], result[1][:,0:n_class])
# np.mean((Y_train - res)**2)
# 0.4011543372347976


In [ ]:
A_prime.shape

In [ ]:
"""
<tf.Tensor: shape=(11, 1), dtype=float64, numpy=
array([[-0.02956712],
       [-0.43500185],
       [ 0.46663761],
       [ 0.28479513],
       [-0.04591487],
       [ 0.22732162],
       [ 0.19878154],
       [-0.21300145],
       [ 0.0071286 ],
       [ 0.11102323],
       [-0.00936144]])>
"""

In [ ]:
for i in range(0, Y_train.shape[0]):
    print(Y_train[i], res[i])

In [62]:
reload(MultiLearn_GLRM)
from MultiLearn_GLRM import Multi_Learn, generate_AXY
from MultiLearn_GLRM.Params.semisupervised_params import Multilearn_GLRM_Semisupervised_Train_Params, Multilearn_GLRM_Semisupervised_Test_Params

A = X_test
A_prime =tf.constant(A)


X = np.random.standard_normal((A.shape[0], embedding_dim))
X_prime = X
X_prime = tf.Variable(X_prime)

Y_prime = Y_final

GLRM_loss_list, X_regularization_loss_list, Y_regularization_loss_list, X_grad_restrictions, Y_grad_restrictions =  Multilearn_GLRM_Semisupervised_Test_Params(A_prime, X_prime, Y_prime)
params["test_hyper_params"] = [GLRM_loss_list, X_regularization_loss_list, Y_regularization_loss_list]

num_iterations=10000
learning_rate=0.004
results_log = []
for i in range(0,10):
    A = X_test
    A_prime =tf.constant(A)


    X = np.random.uniform(-1,1, (A.shape[0], embedding_dim))
    X_prime = X
    X_prime = tf.Variable(X_prime)

    Y_prime = Y_final

    result_val = Multi_Learn.predict(A_prime, X_prime, Y_prime, GLRM_loss_list, X_regularization_loss_list, Y_regularization_loss_list, num_iterations, learning_rate)
    val_pred = np.hstack((result_val[0], A_prime))
    val_pred = np.hstack((val_pred, np.ones(shape=(val_pred.shape[0],1))))
    val_pred = np.matmul(val_pred, beta)
    val_pred = tf.nn.softmax(val_pred).numpy()
    accuracy = accuracy_score(np.argmax(one_hot_targets_val, axis=1), np.argmax(val_pred, axis=1))
    print(np.argmax(val_pred, axis=1))
    print(accuracy)
    results_log.append(accuracy)


params["accuracy"] = results_log

# with open('./logs/Prostate_'+ str(round(np.mean(results_log),5))+'_MSE.json', 'w') as f:
#     f.write(json.dumps(params, sort_keys=True, indent=4, separators=(',', ': ')))




iter: 0 Total loss: tf.Tensor(1.714244190142078, shape=(), dtype=float64)
iter: 50 Total loss: tf.Tensor(1.4350532983377189, shape=(), dtype=float64)
iter: 95 Total loss: tf.Tensor(1.259390872334908, shape=(), dtype=float64)
[1 1 3 1 1 2 1 1 1 1 1 3 1 1 1 1 3 1 3 1]
0.55
iter: 0 Total loss: tf.Tensor(1.7006982448044616, shape=(), dtype=float64)
iter: 50 Total loss: tf.Tensor(1.4254056720686825, shape=(), dtype=float64)
iter: 91 Total loss: tf.Tensor(1.2665382308773843, shape=(), dtype=float64)
[1 1 3 1 1 2 1 1 1 1 1 3 1 1 1 1 3 1 3 1]
0.55
iter: 0 Total loss: tf.Tensor(1.6929334573543664, shape=(), dtype=float64)
iter: 50 Total loss: tf.Tensor(1.4206355510093664, shape=(), dtype=float64)
iter: 90 Total loss: tf.Tensor(1.2668052806772603, shape=(), dtype=float64)
[1 1 3 1 1 2 1 1 1 1 1 3 1 1 1 1 3 1 3 1]
0.55
iter: 0 Total loss: tf.Tensor(1.6710127329308406, shape=(), dtype=float64)
iter: 50 Total loss: tf.Tensor(1.403759955268066, shape=(), dtype=float64)
iter: 88 Total loss: tf.Tensor

KeyboardInterrupt: 

In [29]:
Y_test

array([[3],
       [2],
       [0],
       [4],
       [0],
       [2],
       [1],
       [3],
       [0],
       [4],
       [0],
       [2],
       [0],
       [3],
       [1],
       [3],
       [4],
       [1],
       [2],
       [2],
       [2],
       [4],
       [3],
       [4],
       [3]])

In [ ]:
params

In [ ]:
val_pred = np.hstack((result_val[0], A_prime))
val_pred = np.hstack((val_pred, np.ones(shape=(val_pred.shape[0],1))))
regress_val = np.matmul(val_pred, beta)
np.mean((scaler2.inverse_transform(Y_test) - scaler2.inverse_transform(regress_val))**2)

In [ ]:
regress_val

In [ ]:
Y_train

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test, predictions_val), confusion_matrix(Y_train, predictions)

In [ ]:
409/(409+200), 409/(409+857)

In [ ]:
305/(305+133), 305/(305+961)

In [ ]:
2027/(2027+538), 2027/(2027+3343)

In [ ]:
from sklearn.linear_model import LogisticRegression

X_train_embedding = result[0][:, 1:embedding_dim+1]
X_train_embedding = X_train_embedding.numpy()

X_test_embedding = result_val[0].numpy()


clf = LogisticRegression().fit(X_train_embedding, Y_train)
pred_logistic = clf.predict(X_train_embedding)
pred_logistic_val = clf.predict(X_test_embedding)
accuracy_score(Y_train, pred_logistic), accuracy_score(Y_test, pred_logistic_val)

In [ ]:
confusion_matrix(Y_test, pred_logistic_val), confusion_matrix(Y_train, pred_logistic)

In [ ]:
328/(328+236), 328/(328+938)

In [ ]:
1565/(1565+650), 1565/(1565+3805)

In [ ]:
beta[5:]

In [ ]:
A_prime[0]

In [ ]:
x_embeddings

In [ ]:
prob_val[0:10]

In [ ]:
prob[0:10]

In [ ]:
temp = tf.Variable([1,1,1])
tf.where(temp!=1).shape[0]
# tf.where([True, False, False, True]).numpy()

In [ ]:
np.min(val_pred,axis=0)

In [ ]:
np.min(result[0],axis=0)

In [ ]:
beta

In [ ]:
result[0][3:10,:]

In [ ]:
'beta' in params.keys()